Execute once cell

In [1]:

from torch.utils.data import DataLoader,TensorDataset
import torch.nn as nn
#import data_download
import functions
#import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import torch
import polars as pl
from torch.nn import functional as F
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
dbname='ai_dataset'
dbuser='postgres'
dbpassword='parola'
dbport=5432
dbhost='127.0.0.1'
table_name='tabela'

In [3]:
tickers=["AAPL","MSFT",'SPY','XAUUSD.OANDA','BCO.ICMTRADER']
for i in range(len(tickers)):
    tickers[i]=tickers[i].replace('.','_')
net_functions=functions.Protected_execution


In [4]:
query=net_functions.create_query(tickers[0],table_name)
cursor,conn=net_functions.create_cursor(dbname,dbuser,dbpassword,dbport,dbhost)
data=pl.read_database(query=query,connection=conn)
#data=data.with_columns(pl.col('aapl_adj_close').pct_change().alias('labels'))
#normalize with pandas
#data= data.select((pl.all()-pl.all().mean()) / pl.all().std())

data=data.drop_nulls()
data


aapl_open,aapl_high,aapl_low,aapl_close,aapl_adj_close,aapl_volume
f64,f64,f64,f64,f64,f64
2.585,2.6696,2.5804,2.2722,2.2722,8.0723443e8
2.6832,2.7136,2.6607,2.2789,2.2789,6.196036e8
2.6725,2.675,2.6339,2.2609,2.2609,4.494224e8
2.6875,2.7393,2.6625,2.3193,2.3193,7.044576e8
2.7404,2.7571,2.705,2.3118,2.3118,6.7504077e8
…,…,…,…,…,…
195.18,195.41,192.97,193.6,193.6,3.71228e7
193.61,193.89,192.83,193.05,193.05,2.89193e7
192.49,193.5,191.09,193.15,193.15,4.80877e7


In [5]:
#get data from polars
#labels=data['labels']
train_data=data['aapl_high']
#make polars data into torch tensors
train_data=torch.tensor(train_data)

batch_size=32

In [6]:
#shorten the tensor
shortened_tensor=net_functions.tensor_shortner(train_data, batch_size)
#create batches of 32
image_tensor=net_functions.image_builder(shortened_tensor,batch_size=batch_size)
#create tensor
labels=net_functions.change(image_tensor, batch_size)

#cut the last image from data variable to adjust for change function
image_tensor=image_tensor[:-1]

In [7]:
#make labels and data into tensors
labels=torch.tensor(labels,dtype=torch.int32)
image_tensor=torch.tensor(image_tensor)

#move them to gpu
labels=labels.to(device)
image_tensor=image_tensor.to(device)


In [9]:
#split data with sklearn
train_dataset,test_dataset,train_labels,test_labels=train_test_split(image_tensor,labels,test_size=.1)

#create pythorch datasets
train_dataset=torch.utils.data.TensorDataset(train_dataset,train_labels)
test_dataset=torch.utils.data.TensorDataset(test_dataset,test_labels)
batchsize=32
train_loader=DataLoader(train_dataset,batch_size=batchsize,shuffle=True,drop_last=True)
#test_loader=DataLoader(test_data,batch_size=test_data.tensors[0].shape[0],drop_last=True)
test_loader=DataLoader(test_dataset,batch_size=batchsize,drop_last=True)


In [10]:



def createTheMNISTNet(printtoggle=False):

  class mnistNet(nn.Module):
    def __init__(self, printtoggle=False):
        super().__init__()

        # Input layer for 1D data of size 32
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=2)
        
        
        self.pool=nn.MaxPool1d(kernel_size=2, stride=2)
        # fully-connected layers
        self.fc1 = nn.Linear(512, 256) 
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)  # Assuming 10 output classes
        self.fc5 = nn.Linear(128, 10)  # Assuming 10 output classes

        # toggle for printing out tensor sizes during forward prop
        #self.print = printtoggle
        self.dropout = nn.Dropout(p=0.25)
        # Use GPU if available
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.to(self.device)  # Move the model to the chosen device

    # forward pass
    def forward(self, x):
        # Move input to device
        x = x.to(self.device)

        # Add a singleton dimension for the channel axis
        x = x.unsqueeze(1)


        # Convolution -> relu
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = self.dropout(x)

        # Convolution -> relu
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout(x)
        
        
        
        # Flatten the feature map
        x = x.view(-1, 256*2)
        #print(f'Flattened: {x.shape}') if self.print else None

        # Fully connected layer
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = F.relu(self.fc2(x))
        x = self.dropout(x)

        x = F.relu(self.fc3(x))
        x = self.dropout(x)

        x = F.relu(self.fc4(x))
        x = self.dropout(x)

        x = self.fc5(x)
        return x





  # create the model instance
  net = mnistNet(printtoggle)
  
  # loss function (assuming classification task)
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.Adam(net.parameters(),lr=.01)

  return net,lossfun,optimizer


In [11]:
def function2trainTheModel(numepochs = 50):

  # number of epochs
  numepochs = 1000
  
  # create a new model
  net, lossfun, optimizer = createTheMNISTNet()

  # initialize losses
  losses = torch.zeros(numepochs).to(device)
  trainAcc = []
  testAcc = []
  i=0
  z=0
  # loop over epochs
  for epochi in range(numepochs):
    z+=1
    # initialize batch losses and accuracies
    epochLoss = 0.0
    epochAcc = 0.0
    print(z)
    # loop over training data batches
    for X, y in train_loader:
      i+=1
      #print(i)
      X = X.unsqueeze(1)  # Add a singleton dimension for the channel axis
     
      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat, y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # accumulate batch loss
      epochLoss += loss.item()

      # compute accuracy
      matches = torch.argmax(yHat, axis=1) == y
      accuracyPct = 100 * torch.mean(matches.float())
      epochAcc += accuracyPct.item()

    # end of batch loop

    # average loss and accuracy for the epoch
    epochLoss /= len(train_loader)
    epochAcc /= len(train_loader)

    # append to lists
    losses[epochi] = epochLoss
    trainAcc.append(epochAcc)

    # test accuracy (evaluate on test set)
    test_epochAcc = 0
    total_correct = 0
    total_samples = 0
    
    '''with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)  # Move data to device
            yHat_test = net(X_test)
            matches_test = torch.argmax(yHat_test, dim=1) == y_test
            total_correct += matches_test.sum().item()
            total_samples += y_test.size(0)
    
    test_epochAcc = 100 * total_correct / total_samples
    testAcc.append(test_epochAcc)

     ''' 
    #test_epochAcc = 0.0
    #with torch.no_grad():
      #for X_test, y_test in test_loader:
        #yHat_test = net(X_test)
        #matches_test = torch.argmax(yHat_test, axis=1) == y_test
        #accuracyPct_test = 100 * torch.mean(matches_test.float())
        #test_epochAcc += accuracyPct_test.item()
    #test_epochAcc /= len(test_loader)
    #testAcc.append(test_epochAcc)

  # end epochs loop

  # function output
  return trainAcc, testAcc, losses, net



In [12]:

trainAcc,testAcc,losses,net = function2trainTheModel(20)


1


RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [32, 1, 1, 1, 32]

In [ ]:
nploss=losses.to("cpu")
nploss=nploss.cpu()
nploss=nploss.cpu().numpy()
plt.plot(nploss)
plt.title('losses over time')


In [ ]:
def test(model, device, testloader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            data=data.unsqueeze(1)
            output = model(data)
            test_loss += criterion(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testloader.dataset)
    accuracy = 100. * correct / len(testloader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(testloader.dataset)} '
          f'({accuracy:.0f}%)\n')
    print(nploss)

In [ ]:
test(net,device,test_loader,nn.CrossEntropyLoss())